In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
x=df.drop('label',axis=1)

In [6]:
 y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [7]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
voc_size=10000

In [10]:
messages=x.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [14]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[5549, 9702, 1144, 8320, 7454, 3761, 9328, 5543, 51, 2362],
 [302, 1041, 9105, 1511, 91, 6666, 2129],
 [1131, 5007, 4986, 1212],
 [1506, 900, 7749, 9035, 8948, 9953],
 [7282, 91, 1117, 4940, 6512, 364, 91, 3321, 6909, 5990],
 [4489,
  5456,
  3702,
  206,
  7042,
  51,
  3019,
  2140,
  501,
  8233,
  9893,
  2954,
  5577,
  6527,
  2129],
 [687, 1791, 8654, 6669, 1230, 5530, 4334, 3637, 9029, 3871, 1254],
 [8744, 6090, 317, 5637, 3371, 3677, 51, 1567, 9029, 3871, 1254],
 [9411, 3182, 912, 2746, 3179, 9776, 1967, 7765, 51, 9765],
 [9616, 3476, 6485, 203, 3698, 8332, 1840, 9052],
 [8844, 6406, 290, 9522, 5879, 1513, 4667, 2684, 3524, 2926, 9004],
 [9035, 7618, 7454, 9776, 51, 3371],
 [6682, 689, 5610, 5755, 6485, 6052, 7288, 3474, 6473],
 [8775, 3755, 1304, 7727, 9696, 2846, 2889, 9029, 3871, 1254],
 [4082, 3021, 8058, 1640, 6815, 9029, 3871, 1254],
 [9960, 7650, 5307, 8322, 7395, 5533, 5478, 1392, 8196, 7831],
 [7842, 1063, 1041],
 [1120, 6349, 886, 6884, 51, 2822, 9517, 2129],
 [6641

In [16]:
sent_length=50
embed_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embed_doc

array([[   0,    0,    0, ..., 5543,   51, 2362],
       [   0,    0,    0, ...,   91, 6666, 2129],
       [   0,    0,    0, ..., 5007, 4986, 1212],
       ...,
       [   0,    0,    0, ..., 9029, 3871, 1254],
       [   0,    0,    0, ..., 1176, 2463, 6670],
       [   0,    0,    0, ...,  904, 2646, 6469]])

In [17]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [103]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Conv1D(100,3,padding='valid',activation='tanh'))
model.add(MaxPooling1D())
model.add(Conv1D(10,3,padding='valid',activation='tanh'))
model.add(MaxPooling1D())
#model.add(LSTM(100,activation='relu',return_sequences=True))
model.add(LSTM(100,activation='relu',return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(100,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 50, 40)            400000    
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 48, 100)           12100     
_________________________________________________________________
max_pooling1d_34 (MaxPooling (None, 24, 100)           0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 22, 10)            3010      
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 11, 10)            0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 11, 100)           44400     
_________________________________________________________________
dropout_40 (Dropout)         (None, 11, 100)         

In [104]:
import numpy as np
x_final=np.array(embed_doc)
y_final=np.array(y)

In [105]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.4, random_state=42)

In [108]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=3,batch_size=64)

Epoch 1/3
172/172 [==============================] - 6s 33ms/step - loss: 0.1310 - accuracy: 0.9519 - val_loss: 0.1821 - val_accuracy: 0.9214
Epoch 2/3
172/172 [==============================] - 5s 32ms/step - loss: 0.0560 - accuracy: 0.9828 - val_loss: 0.2509 - val_accuracy: 0.9200
Epoch 3/3
172/172 [==============================] - 5s 31ms/step - loss: 0.0219 - accuracy: 0.9936 - val_loss: 0.3208 - val_accuracy: 0.9200


In [70]:
predict_x=model.predict(x_test) 
pred=np.round(predict_x).astype(int)

In [71]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print("Hybrid CNN-LSTM Data")
score=metrics.accuracy_score(y_test, pred)
print("accuracy: %0.3f" %score)
cm=metrics.confusion_matrix(y_test,pred)
cm

Hybrid CNN-LSTM Data
accuracy: 0.924


array([[3766,  367],
       [ 186, 2995]], dtype=int64)

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      4133
           1       0.89      0.94      0.92      3181

    accuracy                           0.92      7314
   macro avg       0.92      0.93      0.92      7314
weighted avg       0.93      0.92      0.92      7314



In [73]:
from sklearn.metrics import precision_recall_curve
precision,recall,thres = precision_recall_curve(y_test,pred)
tpr=cm[0][0]/(cm[0][0]+cm[1][0])
fpr=cm[0][1]/(cm[0][1]+cm[1][1])
print("tpr:",tpr)
print("fpr:",fpr)

tpr: 0.9529352226720648
fpr: 0.10916121356335515
